In [6]:
#Import the dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [7]:
# Generate a new set of 1,500 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs

In [8]:
# Add the latitudes and longitudes in a list.
coordinates = list(lats_lngs)

In [9]:
#Create a list for holding the names of cities.
cities = []
#Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

775

In [10]:
# Starting URL for weather map API call.
url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [11]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | mataura
Processing Record 2 of Set 1 | mayor pablo lagerenza
Processing Record 3 of Set 1 | podgornoye
Processing Record 4 of Set 1 | lerwick
Processing Record 5 of Set 1 | la asuncion
Processing Record 6 of Set 1 | tiksi
Processing Record 7 of Set 1 | cuamba
Processing Record 8 of Set 1 | yuancheng
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | sarkand
Processing Record 12 of Set 1 | coahuayana
Processing Record 13 of Set 1 | adrar
Processing Record 14 of Set 1 | sinnamary
Processing Record 15 of Set 1 | louisbourg
City not found. Skipping...
Processing Record 16 of Set 1 | tuktoyaktuk
Processing Record 17 of Set 1 | ancud
Processing Record 18 of Set 1 | kavaratti
Processing Record 19 of Set 1 | ribeira grande
Processing Record 20 of Set 1 | hilo
Processing Record 21 of Set 1 | bowen
Processing Record 22 of Set 1 | zhigansk
Processi

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mataura,NZ,-46.1927,168.8643,60.48,39,91,3.83,overcast clouds
1,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,60.28,72,100,5.37,overcast clouds
2,Podgornoye,RU,51.7350,39.1496,43.02,81,75,11.18,broken clouds
3,Lerwick,GB,60.1545,-1.1494,50.00,84,99,22.75,overcast clouds
4,La Asuncion,VE,11.0333,-63.8628,78.69,87,60,12.91,broken clouds
5,Tiksi,RU,71.6872,128.8694,0.19,87,100,15.95,overcast clouds
6,Cuamba,MZ,-14.8031,36.5372,71.42,56,3,2.91,clear sky
7,Yuancheng,CN,36.6520,107.7896,22.33,45,0,21.32,clear sky
8,Ushuaia,AR,-54.8000,-68.3000,40.66,100,90,11.50,light intensity drizzle
9,Rikitea,PF,-23.1203,-134.9692,75.52,83,98,13.85,overcast clouds


In [13]:
len(city_data_df)

711

In [14]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")